In [79]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import confusion_matrix,roc_curve,auc,accuracy_score

pd.options.display.html.table_schema=True
pd.options.display.max_rows=None
pd.options.display.max_columns=None
# %matplotlib inline

In [145]:
data=pd.read_excel('111.xlsx',sheet_name='Лист2')
X=np.array(data.drop('Событие', axis=1))
y=np.array(data['Событие'])

[[2 2 0 ... 0 0 1]
 [1 1 0 ... 0 0 1]
 [2 2 1 ... 0 0 2]
 ...
 [2 2 0 ... 0 0 2]
 [2 2 0 ... 0 0 1]
 [2 2 1 ... 0 0 3]]


In [81]:
rand_forest=RandomForestClassifier(random_state=0, n_estimators=840, max_depth=5).fit(X, y)

In [184]:
importances = [{data.columns[index]: x} for index,x in enumerate(rand_forest.feature_importances_) if x>0.05]
importances_ = []
for col in importances:
    importances_.append( list(col.keys())[0] )
    print(col)

{'Покраснение': 0.06948070012182993}
{'неприятные ощущения в глазу – присутствие – имеется': 0.07206658763845382}
{'конъюнктива ВиН век – изменения – отек': 0.06108336589827186}
{'конъюнктива ВиН век -  характер отделяемого': 0.05970203242649546}


In [83]:
print(rand_forest.score(X,y))
y_pred=rand_forest.predict(X)
pd.crosstab(y,y_pred)

0.9655172413793104


col_0,0,1,2,4
row_0,,,,
0,16,0,0,0
1,0,6,0,0
2,0,0,3,0
3,0,1,0,0
4,0,0,0,3


In [84]:
New_X=data[['Покраснение','неприятные ощущения в глазу – присутствие – имеется','конъюнктива ВиН век – изменения – отек','конъюнктива ВиН век -  характер отделяемого']]
rand_forest1=RandomForestClassifier(random_state=0, n_estimators=840, max_depth=5).fit(New_X, y)
print(rand_forest1.score(New_X,y))

0.8275862068965517


In [187]:
from collections import OrderedDict
interact_data = OrderedDict()
copied = pd.DataFrame(data, columns=importances_)
# copied = copied.drop('Событие', 1)
for col in copied:
    col_data = data[col].unique()
    interact_data[col] =(col_data.min(), col_data.max())

In [188]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual, IntSlider
import ipywidgets as widgets
from IPython.display import display, clear_output
import matplotlib.pyplot as plt

def make_preset(**kwargs):
    predicts = list(kwargs.values())
    predicts = [np.asanyarray(predicts)]
    
    predicted = rand_forest1.predict([predicts[0]])
    return f'Предсказанное событие {predicted[0]}'
    
list_widgets={}

for name in interact_data.keys():
    value = interact_data[name]
    list_widgets[name] = widgets.IntSlider(value=value[0] ,min=value[0],max=value[1],description=name)

interact(make_preset, **list_widgets);

interactive(children=(IntSlider(value=0, description='Покраснение', max=2), IntSlider(value=0, description='не…